# OpenAI TTS Demo
Use OpenAI Assistant to generate a script then convert it into audio scripts


In [19]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [53]:
script = """
Host: Welcome back to 'Health Innovations', the podcast where we dive into the latest research transforming medicine. I'm your host, Alex, and today we're joined by Dr. Adam Ware, lead author of a cutting-edge study that's uncovering new insights in the battle against breast cancer. Dr. Ware, thank you for being here with us.
Dr. Ware: Thank you for having me, Alex. It's a pleasure to share our findings with your audience.
Host: Breast cancer affects millions of women worldwide, yet there are still so many mysteries about its progression. Dr. Ware, your study focuses on a rather unexpected player – the epithelial sodium channel, or ENaC. Can you explain what this channel is and why it's significant?
Dr. Ware: Certainly, Alex. The epithelial sodium channel is traditionally known for its role in regulating sodium absorption in our cells. However, it has been suggested that this channel might also have a role in cancer development and metastasis. In our latest research, we've explored how ENaC expression impacts cell proliferation in breast cancer.
Host: That is fascinating! Normally, when we think about breast cancer, ion channels aren't the first thing that comes to mind. So, what did your study find about ENaC's role?
Dr. Ware: Our study took a deep dive into publicly available data and in vitro experiments with breast cancer cell lines. We discovered that high ENaC expression is associated with less aggressive breast cancer subtypes. More so, it seems that increasing the levels of ENaC decreases cell proliferation, whereas decreasing its activity or expression does the opposite.
Host: It almost sounds like ENaC acts as a kind of cellular brake in these cancer cells. If these findings hold up, could ENaC become a target for future therapies?
Dr. Ware: Precisely. We are cautiously optimistic. Targeting ion channels is not new in pharmacology, but directing this strategy towards cancer treatment, specifically breast cancer, could open a whole new avenue of therapeutic possibilities.
Host: With such potential, what's the next step? What does this mean for breast cancer patients?
Dr. Ware: We're now looking into how ENaC modulates these effects at the molecular level. Understanding its mechanism can lead to the development of drugs that might modulate its activity. And while we're still in the early stages, there's hope that this could translate into more effective treatments for patients down the line.
Host: Incredible work, Dr. Ware. It's clear that unlocking the secrets of cancer will come from exploring every angle, even the ones we never expected. We'll be following your research with great interest. Thanks for sharing your insights with us today.
Dr. Ware: Thank you, Alex. It's through conversations like this that we can spread awareness and hope.

"""

In [59]:
def split_text_by_speaker(text):
    speaker_text = {}

    lines = [line for line in text.strip().split("\n") if line != '']
    for i, line in enumerate(lines):
        if ": " in line:
            speaker, speech = line.split(": ", 1)
            if "Host" in speaker:
                speaker_text[str(i) + '_host'] = speech
            else:
                speaker_text[str(i) + '_guest'] = speech

    return speaker_text

speaker_map = split_text_by_speaker(script)

In [60]:
speaker_map

{'0_host': "Welcome back to 'Health Innovations', the podcast where we dive into the latest research transforming medicine. I'm your host, Alex, and today we're joined by Dr. Adam Ware, lead author of a cutting-edge study that's uncovering new insights in the battle against breast cancer. Dr. Ware, thank you for being here with us.",
 '1_guest': "Thank you for having me, Alex. It's a pleasure to share our findings with your audience.",
 '2_host': "Breast cancer affects millions of women worldwide, yet there are still so many mysteries about its progression. Dr. Ware, your study focuses on a rather unexpected player – the epithelial sodium channel, or ENaC. Can you explain what this channel is and why it's significant?",
 '3_guest': "Certainly, Alex. The epithelial sodium channel is traditionally known for its role in regulating sodium absorption in our cells. However, it has been suggested that this channel might also have a role in cancer development and metastasis. In our latest rese

In [68]:
import openai
import os
import requests
import io
import tempfile

def text_to_audio(file_name, text, voice):
    response = requests.post(
        "https://api.openai.com/v1/audio/speech",
        headers={
            "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
        },
        json={
            "model": "tts-1",
            "input": text,
            "voice": f'{voice if voice else "onyx"}',
        },
    )

    # audio_file_path = f"{file_name}.wav"
    # with open(audio_file_path, "wb") as audio_file:
    #     for chunk in response.iter_content(chunk_size=1024 * 1024):
    #         audio_file.write(chunk)

    # # To play the audio in Jupyter after saving
    # Audio(audio_file_path)
    # Check if the request was successful
    if response.status_code != 200:
        raise Exception("Request failed with status code")
    # ...
    # Create an in-memory bytes buffer
    audio_bytes_io = io.BytesIO()

    # Write audio data to the in-memory bytes buffer
    for chunk in response.iter_content(chunk_size=1024 * 1024):
        audio_bytes_io.write(chunk)

    # Important: Seek to the start of the BytesIO buffer before returning
    audio_bytes_io.seek(0)

    # Save audio to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, prefix=file_name, suffix=".wav") as tmpfile:
        for chunk in response.iter_content(chunk_size=1024 * 1024):
            tmpfile.write(chunk)
        audio_filename = tmpfile.name

    return audio_filename, audio_bytes_io

In [69]:
audio_bytes_combined = io.BytesIO()
temp_files = []
for item in speaker_map.items():
    if 'host' in item[0]:
        audio_filename, audio_bytes_io = text_to_audio(item[0], item[1], "alloy")
    elif 'guest' in item[0]:
        audio_filename, audio_bytes_io = text_to_audio(item[0], item[1], "nova")
    audio_bytes_combined.write(audio_bytes_io.read())
    temp_files.append(audio_filename)
audio_bytes_combined.seek(0)

# Delete all the temporary files after audio combined
for temp_file in temp_files:
    os.remove(temp_file)

AttributeError: 'str' object has no attribute 'close'

In [70]:
# Delete all the temporary files after audio combined
for temp_file in temp_files:
    os.remove(temp_file)

In [73]:
print(os.getcwd())
os.chdir("../")
print(os.getcwd())

/Users/Chris_Pang/Developer/Code_Repository
/Users/Chris_Pang/Developer/Code_Repository/llm_podgen/output


In [76]:
    # Save the combined audio to a file in the output folder
    # Check if the output directory exists, if not, create it
    output_dir = 'dr_ware_interview'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(os.path.join(output_dir, 'openai_tts_combined_audio.wav'), 'wb') as f:
        f.write(audio_bytes_combined.read())
    audio_bytes_combined.seek(0)

0